In [121]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
#matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats
import math 

# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

ds_original = pd.read_csv('dataset_phishing.csv', header=0, delimiter=',',decimal=',')

print("Dimensionalitat del dataset:", ds_original.shape)
print("Dimensionalitat de les entrades X", x.shape)
print("Dimensionalitat de l'atribut Y", y.shape)


#Podem prescindir de la columna de strings amb la url perquè toda la informació que ens pot donar ja queda reflexada en la resta de característiques
dataset = ds_original.drop(columns = "url")

Dimensionalitat del dataset: (11430, 89)
Dimensionalitat de les entrades X (11430, 87)
Dimensionalitat de l'atribut Y (11430,)


In [122]:
# L' objectiu és predir si una url és de phishing o no, per tant la variable dependent serà STATUS.
y = dataset[:]["status"] 
x = dataset.drop(columns = "status")

In [128]:
print(y.describe())
y = y.replace({"phishing" : 1, "legitimate" : 0})

print("\nContingut de Y:", set(y))

print("\nDimensionalitat de X:", x.shape)

count   11430.000
mean        0.500
std         0.500
min         0.000
25%         0.000
50%         0.500
75%         1.000
max         1.000
Name: status, dtype: float64

Contingut de Y: {0, 1}

Dimensionalitat de X: (11430, 87)


Incialment a $Y$ teniem les etiques de "phishing" i "legitimate". Per poder fer regressió ens interessa convertir-les en un valor binari on 1 indica _Phishing_ i 0 no.
També veïem que les mostres estan repartides al 50% entre les dues categoríes (Com s'indica a la descripció de Kaggle).
$X$ està ara format per només dades numèriques, per tant ja es pot començar amb l

In [99]:
x.describe()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
count,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,...,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000,11430.000
mean,61.127,21.090,0.151,2.481,0.998,0.022,0.141,0.162,0.000,0.293,...,0.125,0.776,0.440,0.073,492.532,4062.544,856756.643,0.020,0.534,3.186
std,55.297,10.777,0.358,1.370,2.087,0.155,0.364,0.821,0.000,0.998,...,0.330,0.417,0.496,0.260,814.769,3107.785,1995606.022,0.140,0.499,2.537
min,12.000,4.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,-1.000,-12.000,0.000,0.000,0.000,0.000
25%,33.000,15.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,1.000,0.000,0.000,84.000,972.250,0.000,0.000,0.000,1.000
50%,47.000,19.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,1.000,0.000,0.000,242.000,3993.000,1651.000,0.000,1.000,3.000
75%,71.000,24.000,0.000,3.000,1.000,0.000,0.000,0.000,0.000,0.000,...,0.000,1.000,1.000,0.000,449.000,7026.750,373845.500,0.000,1.000,5.000
max,1641.000,214.000,1.000,24.000,43.000,4.000,3.000,19.000,0.000,19.000,...,1.000,1.000,1.000,1.000,29829.000,12874.000,10767986.000,1.000,1.000,10.000


In [9]:
dataset_full.columnsdescribe

Index(['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens',
       'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
       'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
       'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com',
       'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
       'ratio_digits_host', 'punycode', 'port', 'tld_in_path',
       'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
       'prefix_suffix', 'random_domain', 'shortening_service',
       'path_extension', 'nb_redirection', 'nb_external_redirection',
       'length_words_raw', 'char_repeat', 'shortest_words_raw',
       'shortest_word_host', 'shortest_word_path', 'longest_words_raw',
       'longest_word_host', 'longest_word_path', 'avg_words_raw',
       'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
       'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
       'statistical_report', 

length_hostname: longitud del "nom de la web", inclós (www) i el domini.